In [1]:
%cd ..

c:\Users\saleh\Documents\GitHub\BarCloudTask


In [2]:
from providers import get_provider
from memory import Memory

from prompt import get_system_prompt, Response
from config import INTENTS

In [3]:
class Session:
    def __init__(self, session_id):

        self.session_id = session_id

        self.memory = Memory()

        self.provider = get_provider()

        self.add_message(role="system", msg=get_system_prompt())

    def get_session_chat(self):
        return self.memory.get_chat()

    def add_message(self, role, msg):
        self.memory.add_msg(role=role, msg=msg)

    def call_user_message(self, msg):

        # call the provider and get the response
        self.add_message(role="user", msg=msg)
        response = self.provider.call(self.get_session_chat())

        ai_msg = response["content"]
        self.add_message(role="assistant", msg=ai_msg)
        resp = Response.model_validate_json(ai_msg)

        # select the intent
        selected_intent = None
        for intent in INTENTS:
            if intent['intent'] == resp.intent:
                selected_intent = intent
                break

        query = selected_intent["sql_query"]
        answer = selected_intent["answer"]
        if resp.params:
            query = query.format(**resp.params)
            answer = answer.format(**resp.params)

        return query, answer, response["prompt_tokens"], response["completion_tokens"], response["total_tokens"]

In [4]:
session = Session("session1")

In [5]:
session.call_user_message("What is the total sales for the last month?")

('SELECT c.CustomerName, SUM(sol.Quantity * sol.UnitPrice) AS TotalValue FROM SalesOrders so JOIN SalesOrderLines sol ON so.SOId = sol.SOId JOIN Customers c ON so.CustomerId = c.CustomerId WHERE so.SODate BETWEEN 2026-01-01 AND 2026-01-31 GROUP BY c.CustomerName ORDER BY TotalValue DESC OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY',
 'Top customer between 2026-01-01 and 2026-01-31.',
 2317,
 43,
 2360)

In [6]:
session.call_user_message("what about last 3 months?")

('SELECT c.CustomerName, SUM(sol.Quantity * sol.UnitPrice) AS TotalValue FROM SalesOrders so JOIN SalesOrderLines sol ON so.SOId = sol.SOId JOIN Customers c ON so.CustomerId = c.CustomerId WHERE so.SODate BETWEEN 2025-11-01 AND 2026-01-31 GROUP BY c.CustomerName ORDER BY TotalValue DESC OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY',
 'Top customer between 2025-11-01 and 2026-01-31.',
 2374,
 43,
 2417)